In [2]:
import requests
import datetime
import time
import csv
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier
from scipy.stats import randint
from imblearn.over_sampling import SMOTE
pd.set_option('display.max_columns', None)
plt.style.use('seaborn')
import xgboost as xgb
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
from xgboost import XGBClassifier
import seaborn as sns

Using TensorFlow backend.


In [107]:
models_df = pd.read_csv('models_df.csv')
models_df.drop('Unnamed: 0', axis=1, inplace=True)
models_df

,fight,winner,loser,date,cumultime_w,cumulStr_Att_fighter_w,cumulStr_Att_oponent_w,cumulTD_Att_fighter_w,cumulTD_Att_oponent_w,KDPM_w,KDPM_ratio_w,SLPM_w,SLPM_ratio_w,SAPM_w,SAPM_ratio_w,Str_Acc_w,Str_Def_w,TD_Acc_w,TD_Def_w,SubPM_w,PassPM_w,RevPM_w,Head_SLPM_w,Body_SLPM_w,Leg_SLPM_w,Dist_SLPM_w,Clinch_SLPM_w,Ground_SLPM_w,wins_w,losses_w,cumultime_l,cumulStr_Att_fighter_l,cumulStr_Att_oponent_l,cumulTD_Att_fighter_l,cumulTD_Att_oponent_l,KDPM_l,KDPM_ratio_l,SLPM_l,SLPM_ratio_l,SAPM_l,SAPM_ratio_l,Str_Acc_l,Str_Def_l,TD_Acc_l,TD_Def_l,SubPM_l,PassPM_l,RevPM_l,Head_SLPM_l,Body_SLPM_l,Leg_SLPM_l,Dist_SLPM_l,Clinch_SLPM_l,Ground_SLPM_l,wins_l,losses_l,implied_prob_w,implied_prob_l,prev_w,prev_l,bday_w,height_w,weight_w,reach_w,stance_w,bday_l,height_l,weight_l,reach_l,stance_l,age_w,age_l,weightclass,location
0,Zabit Magomedsharipov vs Calvin Kattar,Zabit-Magomedsharipov,Calvin-Kattar,2019-11-09,30.633333,528.0,457.0,40.0,13.0,0.000000,0.000000,1.220538,0.217774,1.076002,0.171890,0.004445,0.005535,0.058268,0.176282,0.011871,0.061734,0.005022,0.798205,0.177591,0.244741,0.908598,0.124064,0.187875,5,0,24.816667,613.0,668.0,6.0,5.0,0.025539,0.000000,2.357377,0.162282,2.924004,0.305053,0.003273,0.004022,0.000000,0.166667,0.000000,0.000000,0.000000,1.897407,0.209067,0.250903,2.177244,0.033176,0.146957,4,1,0.724638,0.285714,5,5,1991-03-01,73.0,145.0,73.0,Orthodox,1988-03-26,71.0,145.0,72.0,Orthodox,28.0,31.0,Featherweight,"Moscow, Moscow, Russia"
1,Renato Moicano vs Calvin Kattar,Renato-Moicano,Calvin-Kattar,2018-04-07,23.483333,652.0,559.0,3.0,17.0,0.000000,0.000000,1.852035,0.199361,1.611909,0.168629,0.002392,0.004311,0.333333,0.160294,0.000000,0.006083,0.000000,1.141934,0.198139,0.511961,1.815535,0.024333,0.012167,3,1,9.533333,396.0,346.0,3.0,3.0,0.041406,0.000000,4.613413,0.202279,4.809007,0.217600,0.002019,0.003043,0.000000,0.111111,0.000000,0.000000,0.000000,3.786871,0.341895,0.484647,4.320022,0.086361,0.207030,2,0,0.487805,0.500000,4,2,1989-05-21,71.0,145.0,72.0,Orthodox,1988-03-26,71.0,145.0,72.0,Orthodox,28.0,30.0,Featherweight,"Brooklyn, New York, USA"
2,Jeremy Stephens vs Zabit Magomedsharipov,Zabit-Magomedsharipov,Jeremy-Stephens,2019-03-02,23.633333,407.0,328.0,40.0,5.0,0.000000,0.000000,1.334217,0.257494,0.971573,0.150560,0.004933,0.006317,0.058268,0.333333,0.015387,0.080019,0.006510,0.913732,0.163700,0.256784,0.929883,0.160811,0.243522,4,0,162.383333,2666.0,2449.0,93.0,80.0,0.019524,0.012317,1.143799,0.264351,1.102216,0.200448,0.004390,0.006685,0.062361,0.138020,0.015023,0.013380,0.010208,0.662038,0.208852,0.272910,0.813681,0.130730,0.199387,15,14,0.735294,0.270270,4,29,1991-03-01,73.0,145.0,73.0,Orthodox,1986-05-25,69.0,145.0,71.0,Orthodox,28.0,32.0,Featherweight,"Las Vegas, Nevada, USA"
3,Yair Rodriguez vs Jeremy Stephens,Yair-Rodriguez,Jeremy-Stephens,2019-10-18,49.633333,1139.0,853.0,25.0,28.0,0.020591,0.000000,1.960169,0.492457,1.035139,0.171382,0.003145,0.005646,0.122222,0.159864,0.023506,0.035867,0.005757,1.210359,0.415846,0.333963,1.396634,0.162191,0.401344,7,1,169.383333,2795.0,2570.0,101.0,80.0,0.018717,0.011808,1.155568,0.263506,1.091245,0.195622,0.004381,0.006627,0.061135,0.138020,0.014403,0.016200,0.009786,0.670944,0.210341,0.274282,0.830659,0.126171,0.198738,15,15,0.555556,0.454545,8,30,1992-10-06,71.0,145.0,71.0,Orthodox,1986-05-25,69.0,145.0,71.0,Orthodox,27.0,33.0,Featherweight,"Boston, Massachusetts, USA"
4,Yair Rodriguez vs Jeremy Stephens,Yair-Rodriguez,Jeremy-Stephens,2019-10-18,49.633333,1139.0,853.0,25.0,28.0,0.020591,0.000000,1.960169,0.492457,1.035139,0.171382,0.003145,0.005646,0.122222,0.159864,0.023506,0.035867,0.005757,1.210359,0.415846,0.333963,1.396634,0.162191,0.401344,7,1,169.383333,2795.0,2570.0,101.0,80.0,0.018717,0.011808,1.155568,0.263506,1.091245,0.195622,0.004381,0.006627,0.061135,0.138020,0.014403,0.016200,0.009786,0.670944,0.210341,0.274282,0.830659,0.126171,0.198738,15,15,0.555556,0.454545,8,30,1992-10-06,71.0,145.0,71.0,Orthodox,1986-05-25,69.0,145.0,71.0,Orthodox,27.0,33.0,Feathe

In [108]:
# country = []
# for i in list(models_df['location']):
#     country.append(i[-5:])

# models_df['country'] = country

In [109]:
models_df

,fight,winner,loser,date,cumultime_w,cumulStr_Att_fighter_w,cumulStr_Att_oponent_w,cumulTD_Att_fighter_w,cumulTD_Att_oponent_w,KDPM_w,KDPM_ratio_w,SLPM_w,SLPM_ratio_w,SAPM_w,SAPM_ratio_w,Str_Acc_w,Str_Def_w,TD_Acc_w,TD_Def_w,SubPM_w,PassPM_w,RevPM_w,Head_SLPM_w,Body_SLPM_w,Leg_SLPM_w,Dist_SLPM_w,Clinch_SLPM_w,Ground_SLPM_w,wins_w,losses_w,cumultime_l,cumulStr_Att_fighter_l,cumulStr_Att_oponent_l,cumulTD_Att_fighter_l,cumulTD_Att_oponent_l,KDPM_l,KDPM_ratio_l,SLPM_l,SLPM_ratio_l,SAPM_l,SAPM_ratio_l,Str_Acc_l,Str_Def_l,TD_Acc_l,TD_Def_l,SubPM_l,PassPM_l,RevPM_l,Head_SLPM_l,Body_SLPM_l,Leg_SLPM_l,Dist_SLPM_l,Clinch_SLPM_l,Ground_SLPM_l,wins_l,losses_l,implied_prob_w,implied_prob_l,prev_w,prev_l,bday_w,height_w,weight_w,reach_w,stance_w,bday_l,height_l,weight_l,reach_l,stance_l,age_w,age_l,weightclass,location
0,Zabit Magomedsharipov vs Calvin Kattar,Zabit-Magomedsharipov,Calvin-Kattar,2019-11-09,30.633333,528.0,457.0,40.0,13.0,0.000000,0.000000,1.220538,0.217774,1.076002,0.171890,0.004445,0.005535,0.058268,0.176282,0.011871,0.061734,0.005022,0.798205,0.177591,0.244741,0.908598,0.124064,0.187875,5,0,24.816667,613.0,668.0,6.0,5.0,0.025539,0.000000,2.357377,0.162282,2.924004,0.305053,0.003273,0.004022,0.000000,0.166667,0.000000,0.000000,0.000000,1.897407,0.209067,0.250903,2.177244,0.033176,0.146957,4,1,0.724638,0.285714,5,5,1991-03-01,73.0,145.0,73.0,Orthodox,1988-03-26,71.0,145.0,72.0,Orthodox,28.0,31.0,Featherweight,"Moscow, Moscow, Russia"
1,Renato Moicano vs Calvin Kattar,Renato-Moicano,Calvin-Kattar,2018-04-07,23.483333,652.0,559.0,3.0,17.0,0.000000,0.000000,1.852035,0.199361,1.611909,0.168629,0.002392,0.004311,0.333333,0.160294,0.000000,0.006083,0.000000,1.141934,0.198139,0.511961,1.815535,0.024333,0.012167,3,1,9.533333,396.0,346.0,3.0,3.0,0.041406,0.000000,4.613413,0.202279,4.809007,0.217600,0.002019,0.003043,0.000000,0.111111,0.000000,0.000000,0.000000,3.786871,0.341895,0.484647,4.320022,0.086361,0.207030,2,0,0.487805,0.500000,4,2,1989-05-21,71.0,145.0,72.0,Orthodox,1988-03-26,71.0,145.0,72.0,Orthodox,28.0,30.0,Featherweight,"Brooklyn, New York, USA"
2,Jeremy Stephens vs Zabit Magomedsharipov,Zabit-Magomedsharipov,Jeremy-Stephens,2019-03-02,23.633333,407.0,328.0,40.0,5.0,0.000000,0.000000,1.334217,0.257494,0.971573,0.150560,0.004933,0.006317,0.058268,0.333333,0.015387,0.080019,0.006510,0.913732,0.163700,0.256784,0.929883,0.160811,0.243522,4,0,162.383333,2666.0,2449.0,93.0,80.0,0.019524,0.012317,1.143799,0.264351,1.102216,0.200448,0.004390,0.006685,0.062361,0.138020,0.015023,0.013380,0.010208,0.662038,0.208852,0.272910,0.813681,0.130730,0.199387,15,14,0.735294,0.270270,4,29,1991-03-01,73.0,145.0,73.0,Orthodox,1986-05-25,69.0,145.0,71.0,Orthodox,28.0,32.0,Featherweight,"Las Vegas, Nevada, USA"
3,Yair Rodriguez vs Jeremy Stephens,Yair-Rodriguez,Jeremy-Stephens,2019-10-18,49.633333,1139.0,853.0,25.0,28.0,0.020591,0.000000,1.960169,0.492457,1.035139,0.171382,0.003145,0.005646,0.122222,0.159864,0.023506,0.035867,0.005757,1.210359,0.415846,0.333963,1.396634,0.162191,0.401344,7,1,169.383333,2795.0,2570.0,101.0,80.0,0.018717,0.011808,1.155568,0.263506,1.091245,0.195622,0.004381,0.006627,0.061135,0.138020,0.014403,0.016200,0.009786,0.670944,0.210341,0.274282,0.830659,0.126171,0.198738,15,15,0.555556,0.454545,8,30,1992-10-06,71.0,145.0,71.0,Orthodox,1986-05-25,69.0,145.0,71.0,Orthodox,27.0,33.0,Featherweight,"Boston, Massachusetts, USA"
4,Yair Rodriguez vs Jeremy Stephens,Yair-Rodriguez,Jeremy-Stephens,2019-10-18,49.633333,1139.0,853.0,25.0,28.0,0.020591,0.000000,1.960169,0.492457,1.035139,0.171382,0.003145,0.005646,0.122222,0.159864,0.023506,0.035867,0.005757,1.210359,0.415846,0.333963,1.396634,0.162191,0.401344,7,1,169.383333,2795.0,2570.0,101.0,80.0,0.018717,0.011808,1.155568,0.263506,1.091245,0.195622,0.004381,0.006627,0.061135,0.138020,0.014403,0.016200,0.009786,0.670944,0.210341,0.274282,0.830659,0.126171,0.198738,15,15,0.555556,0.454545,8,30,1992-10-06,71.0,145.0,71.0,Orthodox,1986-05-25,69.0,145.0,71.0,Orthodox,27.0,33.0,Feathe

In [110]:
newest = pd.get_dummies(data=models_df, columns=['stance_w', 'stance_l', 'weightclass'])

In [111]:
newest = newest.sample(frac=1)

In [112]:
newest

,fight,winner,loser,date,cumultime_w,cumulStr_Att_fighter_w,cumulStr_Att_oponent_w,cumulTD_Att_fighter_w,cumulTD_Att_oponent_w,KDPM_w,KDPM_ratio_w,SLPM_w,SLPM_ratio_w,SAPM_w,SAPM_ratio_w,Str_Acc_w,Str_Def_w,TD_Acc_w,TD_Def_w,SubPM_w,PassPM_w,RevPM_w,Head_SLPM_w,Body_SLPM_w,Leg_SLPM_w,Dist_SLPM_w,Clinch_SLPM_w,Ground_SLPM_w,wins_w,losses_w,cumultime_l,cumulStr_Att_fighter_l,cumulStr_Att_oponent_l,cumulTD_Att_fighter_l,cumulTD_Att_oponent_l,KDPM_l,KDPM_ratio_l,SLPM_l,SLPM_ratio_l,SAPM_l,SAPM_ratio_l,Str_Acc_l,Str_Def_l,TD_Acc_l,TD_Def_l,SubPM_l,PassPM_l,RevPM_l,Head_SLPM_l,Body_SLPM_l,Leg_SLPM_l,Dist_SLPM_l,Clinch_SLPM_l,Ground_SLPM_l,wins_l,losses_l,implied_prob_w,implied_prob_l,prev_w,prev_l,bday_w,height_w,weight_w,reach_w,bday_l,height_l,weight_l,reach_l,age_w,age_l,location,stance_w_Orthodox,stance_w_Southpaw,stance_w_Switch,stance_l_Orthodox,stance_l_Southpaw,stance_l_Switch,weightclass_Bantamweight,weightclass_Catch Weight,weightclass_Featherweight,weightclass_Flyweight,weightclass_Heavyweight,weightclass_Light Heavyweight,weightclass_Lightweight,weightclass_Middleweight,weightclass_Welterweight,weightclass_Women's Bantamweight,weightclass_Women's Featherweight,weightclass_Women's Flyweight,weightclass_Women's Strawweight
844,Joseph Benavidez vs Henry Cejudo,Joseph-Benavidez,Henry-Cejudo,2016-12-03,74.400000,1558.0,1069.0,51.0,55.0,0.020279,0.000000,1.440197,0.351205,0.801244,0.125695,0.003234,0.007276,0.065244,0.169405,0.014610,0.086978,0.003252,0.772144,0.379258,0.288794,1.086960,0.209524,0.143713,11,2,30.816667,463.0,541.0,21.0,12.0,0.004636,0.000000,0.908188,0.177786,1.011463,0.279529,0.004061,0.005446,0.050794,0.206349,0.000000,0.000000,0.000000,0.498270,0.340519,0.069399,0.611777,0.282504,0.013907,4,1,0.645161,0.364964,13,5,1984-07-31,64.0,125.0,65.0,1987-02-09,64.0,135.0,64.0,32.0,29.0,"Las Vegas, Nevada, USA",0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1314,Roan Carneiro vs Kenny Robertson,Roan-Carneiro,Kenny-Robertson,2016-09-17,29.800000,155.0,201.0,29.0,1.0,0.000000,0.000000,0.515488,0.255569,0.873917,0.242568,0.017895,0.018612,0.104885,1.000000,0.054573,0.100048,0.004794,0.378814,0.078473,0.058200,0.136311,0.113941,0.265236,3,4,36.883333,566.0,481.0,27.0,25.0,0.010042,0.000000,1.457420,0.489382,1.339267,0.155023,0.006524,0.008387,0.091358,0.116190,0.026128,0.050811,0.003873,0.843901,0.405360,0.208159,0.732107,0.410930,0.314382,4,4,0.476190,0.512821,7,8,1978-06-02,71.0,170.0,74.0,1984-02-14,70.0,170.0,74.0,38.0,32.0,"Hidalgo, Texas, USA",1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
900,Abel Trujillo vs John Makdessi,John-Makdessi,Abel-Trujillo,2017-12-16,69.716667,1860.0,1525.0,25.0,11.0,0.027800,0.000000,1.762933,0.229251,1.489179,0.193464,0.002880,0.004450,0.040606,0.363636,0.004835,0.009670,0.000000,0.980088,0.457284,0.325562,1.467497,0.084516,0.210920,7,6,38.233333,445.0,385.0,32.0,46.0,0.004919,0.000000,1.303916,0.344754,0.919921,0.331417,0.008871,0.012313,0.143750,0.081228,0.058132,0.094797,0.000000,0.949727,0.260163,0.094026,0.660062,0.354421,0.289433,6,3,0.370370,0.574713,13,9,1985-05-03,68.0,155.0,68.0,1983-09-18,68.0,155.0,70.0,32.0,34.0,"Winnipeg, Manitoba, Canada",1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
232,Brad Tavares vs Israel Adesanya,Israel-Adesanya,Brad-Tavares,2018-07-06,11.616667,169.0,229.0,16.0,6.0,0.000000,0.000000,1.111179,0.143088,1.535291,0.272940,0.005572,0.004402,0.012500,0.111111,0.000000,0.018646,0.000000,0.493103,0.228505,0.389572,0.831487,0.279693,0.000000,2,0,97.633333,1380.0,1350.0,55.0,66.0,0.016331,0.010242,1.022292,0.245297,0.964086,0.164747,0.005456,0.006769,0.076667,0.161712,0.005290,0.016996,0.004390,0.598167,0.166481,0.257644,0.682982,0.248674,0.090637,12,4,0.523560,0.487805,2,16,1989-07-22,76.0,185.0,80.0,1987-12-21,73.0,185.0,74.0,28.0,30.0,"Las Vegas, Nevada, USA",0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
805,Joseph Benavidez vs Jussier Formiga,Joseph-Benavidez,Jussier-Formiga,2019-06-29,99.716667,2080.0,1569.0,65.0,85.0,0.016563,0.000000,1.390814,0.314162,0.882487,0.139072,0.003213,0.0

In [113]:
newest.columns

Index(['fight', 'winner', 'loser', 'date', 'cumultime_w',
       'cumulStr_Att_fighter_w', 'cumulStr_Att_oponent_w',
       'cumulTD_Att_fighter_w', 'cumulTD_Att_oponent_w', 'KDPM_w',
       'KDPM_ratio_w', 'SLPM_w', 'SLPM_ratio_w', 'SAPM_w', 'SAPM_ratio_w',
       'Str_Acc_w', 'Str_Def_w', 'TD_Acc_w', 'TD_Def_w', 'SubPM_w', 'PassPM_w',
       'RevPM_w', 'Head_SLPM_w', 'Body_SLPM_w', 'Leg_SLPM_w', 'Dist_SLPM_w',
       'Clinch_SLPM_w', 'Ground_SLPM_w', 'wins_w', 'losses_w', 'cumultime_l',
       'cumulStr_Att_fighter_l', 'cumulStr_Att_oponent_l',
       'cumulTD_Att_fighter_l', 'cumulTD_Att_oponent_l', 'KDPM_l',
       'KDPM_ratio_l', 'SLPM_l', 'SLPM_ratio_l', 'SAPM_l', 'SAPM_ratio_l',
       'Str_Acc_l', 'Str_Def_l', 'TD_Acc_l', 'TD_Def_l', 'SubPM_l', 'PassPM_l',
       'RevPM_l', 'Head_SLPM_l', 'Body_SLPM_l', 'Leg_SLPM_l', 'Dist_SLPM_l',
       'Clinch_SLPM_l', 'Ground_SLPM_l', 'wins_l', 'losses_l',
       'implied_prob_w', 'implied_prob_l', 'prev_w', 'prev_l', 'bday_w',
       'heig

In [114]:
winners = newest[['fight', 'date', 'winner', 'cumultime_w',
       'cumulStr_Att_fighter_w', 'cumulStr_Att_oponent_w',
       'cumulTD_Att_fighter_w', 'cumulTD_Att_oponent_w', 'KDPM_w',
       'KDPM_ratio_w', 'SLPM_w', 'SLPM_ratio_w', 'SAPM_w', 'SAPM_ratio_w',
       'Str_Acc_w', 'Str_Def_w', 'TD_Acc_w', 'TD_Def_w', 'SubPM_w', 'PassPM_w',
       'RevPM_w', 'Head_SLPM_w', 'Body_SLPM_w', 'Leg_SLPM_w', 'Dist_SLPM_w',
       'Clinch_SLPM_w', 'Ground_SLPM_w', 'wins_w', 'losses_w', 'implied_prob_w',
       'prev_w', 'height_w', 'weight_w', 'reach_w', 'age_w', 'stance_w_Orthodox',
       'stance_w_Southpaw', 'stance_w_Switch']]

In [115]:
losers = newest[['fight', 'date', 'loser', 'cumultime_l',
       'cumulStr_Att_fighter_l', 'cumulStr_Att_oponent_l',
       'cumulTD_Att_fighter_l', 'cumulTD_Att_oponent_l', 'KDPM_l',
       'KDPM_ratio_l', 'SLPM_l', 'SLPM_ratio_l', 'SAPM_l', 'SAPM_ratio_l',
       'Str_Acc_l', 'Str_Def_l', 'TD_Acc_l', 'TD_Def_l', 'SubPM_l', 'PassPM_l',
       'RevPM_l', 'Head_SLPM_l', 'Body_SLPM_l', 'Leg_SLPM_l', 'Dist_SLPM_l',
       'Clinch_SLPM_l', 'Ground_SLPM_l', 'wins_l', 'losses_l', 'implied_prob_l',
       'prev_l', 'height_l', 'weight_l', 'reach_l', 'age_l', 'stance_l_Orthodox',
       'stance_l_Southpaw', 'stance_l_Switch']]

In [116]:
len(winners)

1572

In [117]:
len(losers)

1572

In [118]:
winners.rename(columns={'winner': 'fighter'}, inplace=True)
losers.rename(columns={'loser': 'fighter'}, inplace=True)

/Users/hmnathel/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [119]:
winners

,fight,date,fighter,cumultime_w,cumulStr_Att_fighter_w,cumulStr_Att_oponent_w,cumulTD_Att_fighter_w,cumulTD_Att_oponent_w,KDPM_w,KDPM_ratio_w,SLPM_w,SLPM_ratio_w,SAPM_w,SAPM_ratio_w,Str_Acc_w,Str_Def_w,TD_Acc_w,TD_Def_w,SubPM_w,PassPM_w,RevPM_w,Head_SLPM_w,Body_SLPM_w,Leg_SLPM_w,Dist_SLPM_w,Clinch_SLPM_w,Ground_SLPM_w,wins_w,losses_w,implied_prob_w,prev_w,height_w,weight_w,reach_w,age_w,stance_w_Orthodox,stance_w_Southpaw,stance_w_Switch
844,Joseph Benavidez vs Henry Cejudo,2016-12-03,Joseph-Benavidez,74.400000,1558.0,1069.0,51.0,55.0,0.020279,0.000000,1.440197,0.351205,0.801244,0.125695,0.003234,0.007276,0.065244,0.169405,0.014610,0.086978,0.003252,0.772144,0.379258,0.288794,1.086960,0.209524,0.143713,11,2,0.645161,13,64.0,125.0,65.0,32.0,0,1,0
1314,Roan Carneiro vs Kenny Robertson,2016-09-17,Roan-Carneiro,29.800000,155.0,201.0,29.0,1.0,0.000000,0.000000,0.515488,0.255569,0.873917,0.242568,0.017895,0.018612,0.104885,1.000000,0.054573,0.100048,0.004794,0.378814,0.078473,0.058200,0.136311,0.113941,0.265236,3,4,0.476190,7,71.0,170.0,74.0,38.0,1,0,0
900,Abel Trujillo vs John Makdessi,2017-12-16,John-Makdessi,69.716667,1860.0,1525.0,25.0,11.0,0.027800,0.000000,1.762933,0.229251,1.489179,0.193464,0.002880,0.004450,0.040606,0.363636,0.004835,0.009670,0.000000,0.980088,0.457284,0.325562,1.467497,0.084516,0.210920,7,6,0.370370,13,68.0,155.0,68.0,32.0,1,0,0
232,Brad Tavares vs Israel Adesanya,2018-07-06,Israel-Adesanya,11.616667,169.0,229.0,16.0,6.0,0.000000,0.000000,1.111179,0.143088,1.535291,0.272940,0.005572,0.004402,0.012500,0.111111,0.000000,0.018646,0.000000,0.493103,0.228505,0.389572,0.831487,0.279693,0.000000,2,0,0.523560,2,76.0,185.0,80.0,28.0,0,0,1
805,Joseph Benavidez vs Jussier Formiga,2019-06-29,Joseph-Benavidez,99.716667,2080.0,1569.0,65.0,85.0,0.016563,0.000000,1.390814,0.314162,0.882487,0.139072,0.003213,0.006444,0.067858,0.137136,0.010901,0.067761,0.005291,0.787324,0.351504,0.251986,1.072896,0.172978,0.144940,14,3,0.534759,17,64.0,125.0,65.0,34.0,0,1,0
798,Randa Markos vs Nina Ansaroff,2018-07-28,Nina-Ansaroff,26.650000,520.0,588.0,4.0,16.0,0.000000,0.000000,1.329497,0.133537,1.583860,0.199543,0.003493,0.003399,0.125000,0.153125,0.010721,0.010721,0.005360,0.764321,0.297389,0.267787,1.127316,0.191460,0.010721,2,2,0.574713,4,65.0,115.0,64.0,32.0,1,0,0
527,Raphael Assuncao vs Marlon Moraes,2019-02-02,Marlon-Moraes,15.666667,272.0,284.0,6.0,3.0,0.123441,0.000000,1.193440,0.275243,1.786294,0.156533,0.004422,0.007950,0.083333,0.500000,0.000000,0.000000,0.000000,0.698589,0.321599,0.173252,1.051762,0.018237,0.123441,3,1,0.689655,4,66.0,135.0,67.0,30.0,1,0,0
1003,Jake Ellenberger vs Mike Perry,2017-04-22,Mike-Perry,17.300000,358.0,397.0,4.0,5.0,0.000000,0.000000,1.135118,0.097936,1.843309,0.396010,0.002866,0.003629,0.000000,0.120000,0.000000,0.000000,0.000000,0.768479,0.093311,0.273328,1.101262,0.033856,0.000000,2,1,0.555556,3,70.0,170.0,71.0,25.0,1,0,0
214,Elias Theodorou vs Cezar Ferreira,2017-02-19,Elias-Theodorou,31.800000,472.0,329.0,21.0,22.0,0.000000,0.000000,0.904613,0.273644,0.807733,0.238517,0.004395,0.008233,0.091837,0.057110,0.000000,0.014518,0.000000,0.417426,0.312847,0.174341,0.477769,0.304862,0.121982,4,1,0.476190,5,73.0,185.0,75.0,28.0,1,0,0
108,Ashley Yoder vs Randa Markos,2019-10-26,Randa-Markos,71.966667,1209.0,961.0,30.0,49.0,0.000000,0.000000,0.980358,0.337053,0.719208,0.120208,0.003900,0.007258,0.095079,0.102742,0.012156,0.049060,0.007013,0.672620,0.114927,0.192810,0.624884,0.175652,0.179823,5,6,0.534759,11,64.0,115.0,63.0,34.0,1,0,0


In [120]:
losers

,fight,date,fighter,cumultime_l,cumulStr_Att_fighter_l,cumulStr_Att_oponent_l,cumulTD_Att_fighter_l,cumulTD_Att_oponent_l,KDPM_l,KDPM_ratio_l,SLPM_l,SLPM_ratio_l,SAPM_l,SAPM_ratio_l,Str_Acc_l,Str_Def_l,TD_Acc_l,TD_Def_l,SubPM_l,PassPM_l,RevPM_l,Head_SLPM_l,Body_SLPM_l,Leg_SLPM_l,Dist_SLPM_l,Clinch_SLPM_l,Ground_SLPM_l,wins_l,losses_l,implied_prob_l,prev_l,height_l,weight_l,reach_l,age_l,stance_l_Orthodox,stance_l_Southpaw,stance_l_Switch
844,Joseph Benavidez vs Henry Cejudo,2016-12-03,Henry-Cejudo,30.816667,463.0,541.0,21.0,12.0,0.004636,0.000000,0.908188,0.177786,1.011463,0.279529,0.004061,0.005446,0.050794,0.206349,0.000000,0.000000,0.000000,0.498270,0.340519,0.069399,0.611777,0.282504,0.013907,4,1,0.364964,5,64.0,135.0,64.0,29.0,1,0,0
1314,Roan Carneiro vs Kenny Robertson,2016-09-17,Kenny-Robertson,36.883333,566.0,481.0,27.0,25.0,0.010042,0.000000,1.457420,0.489382,1.339267,0.155023,0.006524,0.008387,0.091358,0.116190,0.026128,0.050811,0.003873,0.843901,0.405360,0.208159,0.732107,0.410930,0.314382,4,4,0.512821,8,70.0,170.0,74.0,32.0,1,0,0
900,Abel Trujillo vs John Makdessi,2017-12-16,Abel-Trujillo,38.233333,445.0,385.0,32.0,46.0,0.004919,0.000000,1.303916,0.344754,0.919921,0.331417,0.008871,0.012313,0.143750,0.081228,0.058132,0.094797,0.000000,0.949727,0.260163,0.094026,0.660062,0.354421,0.289433,6,3,0.574713,9,68.0,155.0,70.0,34.0,1,0,0
232,Brad Tavares vs Israel Adesanya,2018-07-06,Brad-Tavares,97.633333,1380.0,1350.0,55.0,66.0,0.016331,0.010242,1.022292,0.245297,0.964086,0.164747,0.005456,0.006769,0.076667,0.161712,0.005290,0.016996,0.004390,0.598167,0.166481,0.257644,0.682982,0.248674,0.090637,12,4,0.487805,16,73.0,185.0,74.0,30.0,1,0,0
805,Joseph Benavidez vs Jussier Formiga,2019-06-29,Jussier-Formiga,76.033333,637.0,811.0,48.0,31.0,0.005418,0.000000,0.553940,0.165122,0.644847,0.271019,0.007769,0.010400,0.107292,0.237097,0.017362,0.084553,0.001879,0.380504,0.128354,0.045083,0.320518,0.108263,0.125159,9,4,0.454545,13,65.0,125.0,67.0,34.0,1,0,0
798,Randa Markos vs Nina Ansaroff,2018-07-28,Randa-Markos,53.566667,956.0,775.0,29.0,37.0,0.000000,0.000000,1.064601,0.280046,0.789754,0.119995,0.003735,0.006392,0.063875,0.084466,0.012089,0.054759,0.009422,0.761469,0.113431,0.189701,0.676848,0.190771,0.196981,4,4,0.444444,8,64.0,115.0,63.0,32.0,1,0,0
527,Raphael Assuncao vs Marlon Moraes,2019-02-02,Raphael-Assuncao,73.633333,1480.0,1360.0,39.0,32.0,0.013129,0.000000,1.640035,0.375643,0.914243,0.121946,0.003701,0.006621,0.084829,0.275000,0.006705,0.035467,0.000000,0.845606,0.286904,0.507526,1.332773,0.106255,0.201007,11,2,0.344828,13,65.0,135.0,66.0,36.0,1,0,0
1003,Jake Ellenberger vs Mike Perry,2017-04-22,Jake-Ellenberger,82.500000,979.0,1122.0,36.0,37.0,0.030815,0.036364,0.992694,0.305339,1.238344,0.372646,0.007130,0.009367,0.157870,0.280245,0.012797,0.028721,0.007738,0.717104,0.183440,0.092149,0.590377,0.201424,0.200893,10,8,0.454545,18,69.0,170.0,71.0,32.0,1,0,0
214,Elias Theodorou vs Cezar Ferreira,2017-02-19,Cezar-Ferreira,47.550000,398.0,422.0,18.0,14.0,0.047264,0.000000,0.973714,0.217733,0.726585,0.254870,0.011503,0.014805,0.199074,0.314286,0.033454,0.045068,0.000000,0.755601,0.153826,0.064287,0.649505,0.022842,0.301367,7,3,0.523560,10,73.0,185.0,78.0,32.0,0,1,0
108,Ashley Yoder vs Randa Markos,2019-10-26,Ashley-Yoder,35.000000,557.0,494.0,21.0,8.0,0.000000,0.000000,1.032653,0.193373,0.914286,0.153097,0.004097,0.005317,0.088889,0.083333,0.016327,0.077551,0.016327,0.710204,0.216327,0.106122,0.587755,0.138776,0.306122,2,3,0.454545,5,67.0,115.0,69.0,32.0,0,1,0


In [121]:
winners.columns = [c.replace('_w', '') for c in winners]
losers.columns = [c.replace('_l', '') for c in losers]

In [122]:
fighter1 = pd.DataFrame()
fighter1 = fighter1.append(winners[0:786])
fighter1 = fighter1.append(losers[786:])

In [123]:
fighter1

,fight,date,fighter,cumultime,cumulStr_Att_fighter,cumulStr_Att_oponent,cumulTD_Att_fighter,cumulTD_Att_oponent,KDPM,KDPM_ratio,SLPM,SLPM_ratio,SAPM,SAPM_ratio,Str_Acc,Str_Def,TD_Acc,TD_Def,SubPM,PassPM,RevPM,Head_SLPM,Body_SLPM,Leg_SLPM,Dist_SLPM,Clinch_SLPM,Ground_SLPM,wins,losses,implied_prob,prev,height,weight,reach,age,stance_Orthodox,stance_Southpaw,stance_Switch
844,Joseph Benavidez vs Henry Cejudo,2016-12-03,Joseph-Benavidez,74.400000,1558.0,1069.0,51.0,55.0,0.020279,0.000000,1.440197,0.351205,0.801244,0.125695,0.003234,0.007276,0.065244,0.169405,0.014610,0.086978,0.003252,0.772144,0.379258,0.288794,1.086960,0.209524,0.143713,11,2,0.645161,13,64.0,125.0,65.0,32.0,0,1,0
1314,Roan Carneiro vs Kenny Robertson,2016-09-17,Roan-Carneiro,29.800000,155.0,201.0,29.0,1.0,0.000000,0.000000,0.515488,0.255569,0.873917,0.242568,0.017895,0.018612,0.104885,1.000000,0.054573,0.100048,0.004794,0.378814,0.078473,0.058200,0.136311,0.113941,0.265236,3,4,0.476190,7,71.0,170.0,74.0,38.0,1,0,0
900,Abel Trujillo vs John Makdessi,2017-12-16,John-Makdessi,69.716667,1860.0,1525.0,25.0,11.0,0.027800,0.000000,1.762933,0.229251,1.489179,0.193464,0.002880,0.004450,0.040606,0.363636,0.004835,0.009670,0.000000,0.980088,0.457284,0.325562,1.467497,0.084516,0.210920,7,6,0.370370,13,68.0,155.0,68.0,32.0,1,0,0
232,Brad Tavares vs Israel Adesanya,2018-07-06,Israel-Adesanya,11.616667,169.0,229.0,16.0,6.0,0.000000,0.000000,1.111179,0.143088,1.535291,0.272940,0.005572,0.004402,0.012500,0.111111,0.000000,0.018646,0.000000,0.493103,0.228505,0.389572,0.831487,0.279693,0.000000,2,0,0.523560,2,76.0,185.0,80.0,28.0,0,0,1
805,Joseph Benavidez vs Jussier Formiga,2019-06-29,Joseph-Benavidez,99.716667,2080.0,1569.0,65.0,85.0,0.016563,0.000000,1.390814,0.314162,0.882487,0.139072,0.003213,0.006444,0.067858,0.137136,0.010901,0.067761,0.005291,0.787324,0.351504,0.251986,1.072896,0.172978,0.144940,14,3,0.534759,17,64.0,125.0,65.0,34.0,0,1,0
798,Randa Markos vs Nina Ansaroff,2018-07-28,Nina-Ansaroff,26.650000,520.0,588.0,4.0,16.0,0.000000,0.000000,1.329497,0.133537,1.583860,0.199543,0.003493,0.003399,0.125000,0.153125,0.010721,0.010721,0.005360,0.764321,0.297389,0.267787,1.127316,0.191460,0.010721,2,2,0.574713,4,65.0,115.0,64.0,32.0,1,0,0
527,Raphael Assuncao vs Marlon Moraes,2019-02-02,Marlon-Moraes,15.666667,272.0,284.0,6.0,3.0,0.123441,0.000000,1.193440,0.275243,1.786294,0.156533,0.004422,0.007950,0.083333,0.500000,0.000000,0.000000,0.000000,0.698589,0.321599,0.173252,1.051762,0.018237,0.123441,3,1,0.689655,4,66.0,135.0,67.0,30.0,1,0,0
1003,Jake Ellenberger vs Mike Perry,2017-04-22,Mike-Perry,17.300000,358.0,397.0,4.0,5.0,0.000000,0.000000,1.135118,0.097936,1.843309,0.396010,0.002866,0.003629,0.000000,0.120000,0.000000,0.000000,0.000000,0.768479,0.093311,0.273328,1.101262,0.033856,0.000000,2,1,0.555556,3,70.0,170.0,71.0,25.0,1,0,0
214,Elias Theodorou vs Cezar Ferreira,2017-02-19,Elias-Theodorou,31.800000,472.0,329.0,21.0,22.0,0.000000,0.000000,0.904613,0.273644,0.807733,0.238517,0.004395,0.008233,0.091837,0.057110,0.000000,0.014518,0.000000,0.417426,0.312847,0.174341,0.477769,0.304862,0.121982,4,1,0.476190,5,73.0,185.0,75.0,28.0,1,0,0
108,Ashley Yoder vs Randa Markos,2019-10-26,Randa-Markos,71.966667,1209.0,961.0,30.0,49.0,0.000000,0.000000,0.980358,0.337053,0.719208,0.120208,0.003900,0.007258,0.095079,0.102742,0.012156,0.049060,0.007013,0.672620,0.114927,0.192810,0.624884,0.175652,0.179823,5,6,0.534759,11,64.0,115.0,63.0,34.0,1,0,0


In [124]:
fighter2 = pd.DataFrame()
fighter2 = fighter2.append(losers[0:786])
fighter2 = fighter2.append(winners[786:])

In [125]:
fighter2

,fight,date,fighter,cumultime,cumulStr_Att_fighter,cumulStr_Att_oponent,cumulTD_Att_fighter,cumulTD_Att_oponent,KDPM,KDPM_ratio,SLPM,SLPM_ratio,SAPM,SAPM_ratio,Str_Acc,Str_Def,TD_Acc,TD_Def,SubPM,PassPM,RevPM,Head_SLPM,Body_SLPM,Leg_SLPM,Dist_SLPM,Clinch_SLPM,Ground_SLPM,wins,losses,implied_prob,prev,height,weight,reach,age,stance_Orthodox,stance_Southpaw,stance_Switch
844,Joseph Benavidez vs Henry Cejudo,2016-12-03,Henry-Cejudo,30.816667,463.0,541.0,21.0,12.0,0.004636,0.000000,0.908188,0.177786,1.011463,0.279529,0.004061,0.005446,0.050794,0.206349,0.000000,0.000000,0.000000,0.498270,0.340519,0.069399,0.611777,0.282504,0.013907,4,1,0.364964,5,64.0,135.0,64.0,29.0,1,0,0
1314,Roan Carneiro vs Kenny Robertson,2016-09-17,Kenny-Robertson,36.883333,566.0,481.0,27.0,25.0,0.010042,0.000000,1.457420,0.489382,1.339267,0.155023,0.006524,0.008387,0.091358,0.116190,0.026128,0.050811,0.003873,0.843901,0.405360,0.208159,0.732107,0.410930,0.314382,4,4,0.512821,8,70.0,170.0,74.0,32.0,1,0,0
900,Abel Trujillo vs John Makdessi,2017-12-16,Abel-Trujillo,38.233333,445.0,385.0,32.0,46.0,0.004919,0.000000,1.303916,0.344754,0.919921,0.331417,0.008871,0.012313,0.143750,0.081228,0.058132,0.094797,0.000000,0.949727,0.260163,0.094026,0.660062,0.354421,0.289433,6,3,0.574713,9,68.0,155.0,70.0,34.0,1,0,0
232,Brad Tavares vs Israel Adesanya,2018-07-06,Brad-Tavares,97.633333,1380.0,1350.0,55.0,66.0,0.016331,0.010242,1.022292,0.245297,0.964086,0.164747,0.005456,0.006769,0.076667,0.161712,0.005290,0.016996,0.004390,0.598167,0.166481,0.257644,0.682982,0.248674,0.090637,12,4,0.487805,16,73.0,185.0,74.0,30.0,1,0,0
805,Joseph Benavidez vs Jussier Formiga,2019-06-29,Jussier-Formiga,76.033333,637.0,811.0,48.0,31.0,0.005418,0.000000,0.553940,0.165122,0.644847,0.271019,0.007769,0.010400,0.107292,0.237097,0.017362,0.084553,0.001879,0.380504,0.128354,0.045083,0.320518,0.108263,0.125159,9,4,0.454545,13,65.0,125.0,67.0,34.0,1,0,0
798,Randa Markos vs Nina Ansaroff,2018-07-28,Randa-Markos,53.566667,956.0,775.0,29.0,37.0,0.000000,0.000000,1.064601,0.280046,0.789754,0.119995,0.003735,0.006392,0.063875,0.084466,0.012089,0.054759,0.009422,0.761469,0.113431,0.189701,0.676848,0.190771,0.196981,4,4,0.444444,8,64.0,115.0,63.0,32.0,1,0,0
527,Raphael Assuncao vs Marlon Moraes,2019-02-02,Raphael-Assuncao,73.633333,1480.0,1360.0,39.0,32.0,0.013129,0.000000,1.640035,0.375643,0.914243,0.121946,0.003701,0.006621,0.084829,0.275000,0.006705,0.035467,0.000000,0.845606,0.286904,0.507526,1.332773,0.106255,0.201007,11,2,0.344828,13,65.0,135.0,66.0,36.0,1,0,0
1003,Jake Ellenberger vs Mike Perry,2017-04-22,Jake-Ellenberger,82.500000,979.0,1122.0,36.0,37.0,0.030815,0.036364,0.992694,0.305339,1.238344,0.372646,0.007130,0.009367,0.157870,0.280245,0.012797,0.028721,0.007738,0.717104,0.183440,0.092149,0.590377,0.201424,0.200893,10,8,0.454545,18,69.0,170.0,71.0,32.0,1,0,0
214,Elias Theodorou vs Cezar Ferreira,2017-02-19,Cezar-Ferreira,47.550000,398.0,422.0,18.0,14.0,0.047264,0.000000,0.973714,0.217733,0.726585,0.254870,0.011503,0.014805,0.199074,0.314286,0.033454,0.045068,0.000000,0.755601,0.153826,0.064287,0.649505,0.022842,0.301367,7,3,0.523560,10,73.0,185.0,78.0,32.0,0,1,0
108,Ashley Yoder vs Randa Markos,2019-10-26,Ashley-Yoder,35.000000,557.0,494.0,21.0,8.0,0.000000,0.000000,1.032653,0.193373,0.914286,0.153097,0.004097,0.005317,0.088889,0.083333,0.016327,0.077551,0.016327,0.710204,0.216327,0.106122,0.587755,0.138776,0.306122,2,3,0.454545,5,67.0,115.0,69.0,32.0,0,1,0


In [126]:
fighter1 = fighter1.add_suffix('_1')
fighter2 = fighter2.add_suffix('_2')

In [127]:
fighter1.rename(columns={'date_1': 'date'}, inplace=True)
fighter2.rename(columns={'date_2': 'date'}, inplace=True)

fighter1.rename(columns={'fight_1': 'fight'}, inplace=True)
fighter2.rename(columns={'fight_2': 'fight'}, inplace=True)

In [128]:
fighter1

,fight,date,fighter_1,cumultime_1,cumulStr_Att_fighter_1,cumulStr_Att_oponent_1,cumulTD_Att_fighter_1,cumulTD_Att_oponent_1,KDPM_1,KDPM_ratio_1,SLPM_1,SLPM_ratio_1,SAPM_1,SAPM_ratio_1,Str_Acc_1,Str_Def_1,TD_Acc_1,TD_Def_1,SubPM_1,PassPM_1,RevPM_1,Head_SLPM_1,Body_SLPM_1,Leg_SLPM_1,Dist_SLPM_1,Clinch_SLPM_1,Ground_SLPM_1,wins_1,losses_1,implied_prob_1,prev_1,height_1,weight_1,reach_1,age_1,stance_Orthodox_1,stance_Southpaw_1,stance_Switch_1
844,Joseph Benavidez vs Henry Cejudo,2016-12-03,Joseph-Benavidez,74.400000,1558.0,1069.0,51.0,55.0,0.020279,0.000000,1.440197,0.351205,0.801244,0.125695,0.003234,0.007276,0.065244,0.169405,0.014610,0.086978,0.003252,0.772144,0.379258,0.288794,1.086960,0.209524,0.143713,11,2,0.645161,13,64.0,125.0,65.0,32.0,0,1,0
1314,Roan Carneiro vs Kenny Robertson,2016-09-17,Roan-Carneiro,29.800000,155.0,201.0,29.0,1.0,0.000000,0.000000,0.515488,0.255569,0.873917,0.242568,0.017895,0.018612,0.104885,1.000000,0.054573,0.100048,0.004794,0.378814,0.078473,0.058200,0.136311,0.113941,0.265236,3,4,0.476190,7,71.0,170.0,74.0,38.0,1,0,0
900,Abel Trujillo vs John Makdessi,2017-12-16,John-Makdessi,69.716667,1860.0,1525.0,25.0,11.0,0.027800,0.000000,1.762933,0.229251,1.489179,0.193464,0.002880,0.004450,0.040606,0.363636,0.004835,0.009670,0.000000,0.980088,0.457284,0.325562,1.467497,0.084516,0.210920,7,6,0.370370,13,68.0,155.0,68.0,32.0,1,0,0
232,Brad Tavares vs Israel Adesanya,2018-07-06,Israel-Adesanya,11.616667,169.0,229.0,16.0,6.0,0.000000,0.000000,1.111179,0.143088,1.535291,0.272940,0.005572,0.004402,0.012500,0.111111,0.000000,0.018646,0.000000,0.493103,0.228505,0.389572,0.831487,0.279693,0.000000,2,0,0.523560,2,76.0,185.0,80.0,28.0,0,0,1
805,Joseph Benavidez vs Jussier Formiga,2019-06-29,Joseph-Benavidez,99.716667,2080.0,1569.0,65.0,85.0,0.016563,0.000000,1.390814,0.314162,0.882487,0.139072,0.003213,0.006444,0.067858,0.137136,0.010901,0.067761,0.005291,0.787324,0.351504,0.251986,1.072896,0.172978,0.144940,14,3,0.534759,17,64.0,125.0,65.0,34.0,0,1,0
798,Randa Markos vs Nina Ansaroff,2018-07-28,Nina-Ansaroff,26.650000,520.0,588.0,4.0,16.0,0.000000,0.000000,1.329497,0.133537,1.583860,0.199543,0.003493,0.003399,0.125000,0.153125,0.010721,0.010721,0.005360,0.764321,0.297389,0.267787,1.127316,0.191460,0.010721,2,2,0.574713,4,65.0,115.0,64.0,32.0,1,0,0
527,Raphael Assuncao vs Marlon Moraes,2019-02-02,Marlon-Moraes,15.666667,272.0,284.0,6.0,3.0,0.123441,0.000000,1.193440,0.275243,1.786294,0.156533,0.004422,0.007950,0.083333,0.500000,0.000000,0.000000,0.000000,0.698589,0.321599,0.173252,1.051762,0.018237,0.123441,3,1,0.689655,4,66.0,135.0,67.0,30.0,1,0,0
1003,Jake Ellenberger vs Mike Perry,2017-04-22,Mike-Perry,17.300000,358.0,397.0,4.0,5.0,0.000000,0.000000,1.135118,0.097936,1.843309,0.396010,0.002866,0.003629,0.000000,0.120000,0.000000,0.000000,0.000000,0.768479,0.093311,0.273328,1.101262,0.033856,0.000000,2,1,0.555556,3,70.0,170.0,71.0,25.0,1,0,0
214,Elias Theodorou vs Cezar Ferreira,2017-02-19,Elias-Theodorou,31.800000,472.0,329.0,21.0,22.0,0.000000,0.000000,0.904613,0.273644,0.807733,0.238517,0.004395,0.008233,0.091837,0.057110,0.000000,0.014518,0.000000,0.417426,0.312847,0.174341,0.477769,0.304862,0.121982,4,1,0.476190,5,73.0,185.0,75.0,28.0,1,0,0
108,Ashley Yoder vs Randa Markos,2019-10-26,Randa-Markos,71.966667,1209.0,961.0,30.0,49.0,0.000000,0.000000,0.980358,0.337053,0.719208,0.120208,0.003900,0.007258,0.095079,0.102742,0.012156,0.049060,0.007013,0.672620,0.114927,0.192810,0.624884,0.175652,0.179823,5,6,0.534759,11,64.0,115.0,63.0,34.0,1,0,0


In [129]:
fighter2

,fight,date,fighter_2,cumultime_2,cumulStr_Att_fighter_2,cumulStr_Att_oponent_2,cumulTD_Att_fighter_2,cumulTD_Att_oponent_2,KDPM_2,KDPM_ratio_2,SLPM_2,SLPM_ratio_2,SAPM_2,SAPM_ratio_2,Str_Acc_2,Str_Def_2,TD_Acc_2,TD_Def_2,SubPM_2,PassPM_2,RevPM_2,Head_SLPM_2,Body_SLPM_2,Leg_SLPM_2,Dist_SLPM_2,Clinch_SLPM_2,Ground_SLPM_2,wins_2,losses_2,implied_prob_2,prev_2,height_2,weight_2,reach_2,age_2,stance_Orthodox_2,stance_Southpaw_2,stance_Switch_2
844,Joseph Benavidez vs Henry Cejudo,2016-12-03,Henry-Cejudo,30.816667,463.0,541.0,21.0,12.0,0.004636,0.000000,0.908188,0.177786,1.011463,0.279529,0.004061,0.005446,0.050794,0.206349,0.000000,0.000000,0.000000,0.498270,0.340519,0.069399,0.611777,0.282504,0.013907,4,1,0.364964,5,64.0,135.0,64.0,29.0,1,0,0
1314,Roan Carneiro vs Kenny Robertson,2016-09-17,Kenny-Robertson,36.883333,566.0,481.0,27.0,25.0,0.010042,0.000000,1.457420,0.489382,1.339267,0.155023,0.006524,0.008387,0.091358,0.116190,0.026128,0.050811,0.003873,0.843901,0.405360,0.208159,0.732107,0.410930,0.314382,4,4,0.512821,8,70.0,170.0,74.0,32.0,1,0,0
900,Abel Trujillo vs John Makdessi,2017-12-16,Abel-Trujillo,38.233333,445.0,385.0,32.0,46.0,0.004919,0.000000,1.303916,0.344754,0.919921,0.331417,0.008871,0.012313,0.143750,0.081228,0.058132,0.094797,0.000000,0.949727,0.260163,0.094026,0.660062,0.354421,0.289433,6,3,0.574713,9,68.0,155.0,70.0,34.0,1,0,0
232,Brad Tavares vs Israel Adesanya,2018-07-06,Brad-Tavares,97.633333,1380.0,1350.0,55.0,66.0,0.016331,0.010242,1.022292,0.245297,0.964086,0.164747,0.005456,0.006769,0.076667,0.161712,0.005290,0.016996,0.004390,0.598167,0.166481,0.257644,0.682982,0.248674,0.090637,12,4,0.487805,16,73.0,185.0,74.0,30.0,1,0,0
805,Joseph Benavidez vs Jussier Formiga,2019-06-29,Jussier-Formiga,76.033333,637.0,811.0,48.0,31.0,0.005418,0.000000,0.553940,0.165122,0.644847,0.271019,0.007769,0.010400,0.107292,0.237097,0.017362,0.084553,0.001879,0.380504,0.128354,0.045083,0.320518,0.108263,0.125159,9,4,0.454545,13,65.0,125.0,67.0,34.0,1,0,0
798,Randa Markos vs Nina Ansaroff,2018-07-28,Randa-Markos,53.566667,956.0,775.0,29.0,37.0,0.000000,0.000000,1.064601,0.280046,0.789754,0.119995,0.003735,0.006392,0.063875,0.084466,0.012089,0.054759,0.009422,0.761469,0.113431,0.189701,0.676848,0.190771,0.196981,4,4,0.444444,8,64.0,115.0,63.0,32.0,1,0,0
527,Raphael Assuncao vs Marlon Moraes,2019-02-02,Raphael-Assuncao,73.633333,1480.0,1360.0,39.0,32.0,0.013129,0.000000,1.640035,0.375643,0.914243,0.121946,0.003701,0.006621,0.084829,0.275000,0.006705,0.035467,0.000000,0.845606,0.286904,0.507526,1.332773,0.106255,0.201007,11,2,0.344828,13,65.0,135.0,66.0,36.0,1,0,0
1003,Jake Ellenberger vs Mike Perry,2017-04-22,Jake-Ellenberger,82.500000,979.0,1122.0,36.0,37.0,0.030815,0.036364,0.992694,0.305339,1.238344,0.372646,0.007130,0.009367,0.157870,0.280245,0.012797,0.028721,0.007738,0.717104,0.183440,0.092149,0.590377,0.201424,0.200893,10,8,0.454545,18,69.0,170.0,71.0,32.0,1,0,0
214,Elias Theodorou vs Cezar Ferreira,2017-02-19,Cezar-Ferreira,47.550000,398.0,422.0,18.0,14.0,0.047264,0.000000,0.973714,0.217733,0.726585,0.254870,0.011503,0.014805,0.199074,0.314286,0.033454,0.045068,0.000000,0.755601,0.153826,0.064287,0.649505,0.022842,0.301367,7,3,0.523560,10,73.0,185.0,78.0,32.0,0,1,0
108,Ashley Yoder vs Randa Markos,2019-10-26,Ashley-Yoder,35.000000,557.0,494.0,21.0,8.0,0.000000,0.000000,1.032653,0.193373,0.914286,0.153097,0.004097,0.005317,0.088889,0.083333,0.016327,0.077551,0.016327,0.710204,0.216327,0.106122,0.587755,0.138776,0.306122,2,3,0.454545,5,67.0,115.0,69.0,32.0,0,1,0


In [130]:

w1 = [1] * 786
w2 = [2] * 786
w = w1 + w2

In [131]:

len(w)

1572

In [132]:
fighter1['winner'] = w
fighter2['winner'] = w

In [133]:
fighter1.to_csv('fighter1.csv')
fighter2.to_csv('fighter2.csv')